In [30]:
import pandas as pd
data = pd.read_csv("D:/Data Science/POC/Topic Modelling/million-headlines/abcnews-date-text.csv", error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [31]:
print(len(documents))
print(documents[:5])

1103665
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [32]:
##Text preprocessing
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
##from nltk.stem import PorterStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
from nltk.stem.porter import *
stemmer = PorterStemmer()
np.random.seed(2018)

In [33]:
##Lemmatise and Stem pipeline 
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [34]:

doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [35]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [36]:
###WORD DICTIONARY/TOKEN DICTIONARY
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [37]:
##Feature reduction in mo of words from the corpus based on the occurence in documents
##Filter out tokens that appear in

##less than 15 documents (absolute number) or
##more than 0.5 documents (fraction of total corpus size, not absolute number).
##after the above two steps, keep only the first 100000 most frequent tokens.
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [38]:
##Use Gensim doc2bow for storing the words and frequeny of words
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (483, 1), (4021, 1)]

In [39]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4021 ("dampen") appears 1 time.


In [40]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [41]:
##Create tf-idf model object using models.TfidfModel on ‘bow_corpus’
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5903602896750699),
 (1, 0.38524510107363613),
 (2, 0.4974556071174764),
 (3, 0.5055678583740412)]


In [42]:
##Running LDA using Bag of Words
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [43]:
import pickle
filename = "D:/Data Science/POC/Topic Modelling/million-headlines/finalized_model.sav"
pickle.dump(lda_model, open(filename, 'wb'))
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.032*"queensland" + 0.020*"countri" + 0.018*"hospit" + 0.017*"tasmanian" + 0.017*"hour" + 0.015*"work" + 0.014*"children" + 0.013*"citi" + 0.011*"prison" + 0.010*"young"
Topic: 1 
Words: 0.030*"australia" + 0.023*"world" + 0.017*"market" + 0.014*"sydney" + 0.012*"record" + 0.012*"share" + 0.011*"victoria" + 0.009*"australian" + 0.009*"port" + 0.008*"drum"
Topic: 2 
Words: 0.035*"trump" + 0.022*"kill" + 0.021*"north" + 0.020*"coast" + 0.016*"china" + 0.016*"attack" + 0.014*"adelaid" + 0.014*"west" + 0.013*"price" + 0.013*"gold"
Topic: 3 
Words: 0.027*"govern" + 0.017*"plan" + 0.014*"rural" + 0.013*"say" + 0.013*"council" + 0.012*"indigen" + 0.012*"turnbul" + 0.012*"water" + 0.012*"chang" + 0.011*"commun"
Topic: 4 
Words: 0.017*"tasmania" + 0.015*"time" + 0.015*"fight" + 0.013*"lose" + 0.012*"leagu" + 0.010*"john" + 0.010*"life" + 0.009*"premier" + 0.009*"unit" + 0.008*"make"
Topic: 5 
Words: 0.027*"elect" + 0.023*"south" + 0.022*"nation" + 0.016*"open" + 0.014*"austral

In [44]:
##LDA using TFIDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
filename1 = "D:/Data Science/POC/Topic Modelling/million-headlines/finalized_model_lda_tfidf.sav"
pickle.dump(lda_model, open(filename1, 'wb'))
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.012*"drum" + 0.007*"climat" + 0.007*"septemb" + 0.007*"rise" + 0.006*"august" + 0.006*"malcolm" + 0.006*"rate" + 0.006*"rat" + 0.006*"price" + 0.005*"tree"
Topic: 1 Word: 0.017*"charg" + 0.017*"polic" + 0.015*"murder" + 0.011*"court" + 0.010*"death" + 0.010*"woman" + 0.009*"jail" + 0.008*"child" + 0.008*"assault" + 0.008*"accus"
Topic: 2 Word: 0.018*"trump" + 0.011*"market" + 0.009*"donald" + 0.008*"weather" + 0.008*"share" + 0.006*"australian" + 0.006*"street" + 0.006*"tuesday" + 0.006*"live" + 0.006*"wall"
Topic: 3 Word: 0.016*"rural" + 0.016*"countri" + 0.014*"hour" + 0.008*"govern" + 0.008*"health" + 0.007*"fund" + 0.006*"grandstand" + 0.005*"sport" + 0.005*"nation" + 0.005*"servic"
Topic: 4 Word: 0.015*"interview" + 0.007*"australia" + 0.006*"miss" + 0.006*"leagu" + 0.006*"search" + 0.006*"rugbi" + 0.006*"world" + 0.006*"novemb" + 0.006*"test" + 0.005*"sexual"
Topic: 5 Word: 0.008*"dollar" + 0.007*"juli" + 0.007*"thursday" + 0.005*"social" + 0.005*"australian" + 0

In [45]:
##Classification of our test docs
processed_docs[4310]
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.420001357793808	 
Topic: 0.023*"hous" + 0.019*"die" + 0.016*"home" + 0.014*"crash" + 0.014*"break" + 0.013*"take" + 0.013*"perth" + 0.011*"lead" + 0.010*"near" + 0.009*"road"

Score: 0.41999563574790955	 
Topic: 0.027*"govern" + 0.017*"plan" + 0.014*"rural" + 0.013*"say" + 0.013*"council" + 0.012*"indigen" + 0.012*"turnbul" + 0.012*"water" + 0.012*"chang" + 0.011*"commun"

Score: 0.020001839846372604	 
Topic: 0.017*"brisban" + 0.016*"famili" + 0.014*"child" + 0.013*"report" + 0.012*"jail" + 0.011*"abus" + 0.011*"sentenc" + 0.011*"show" + 0.010*"victim" + 0.010*"releas"

Score: 0.020000966265797615	 
Topic: 0.032*"polic" + 0.018*"interview" + 0.018*"health" + 0.018*"miss" + 0.014*"servic" + 0.014*"investig" + 0.013*"concern" + 0.011*"call" + 0.011*"search" + 0.011*"minist"

Score: 0.020000217482447624	 
Topic: 0.035*"trump" + 0.022*"kill" + 0.021*"north" + 0.020*"coast" + 0.016*"china" + 0.016*"attack" + 0.014*"adelaid" + 0.014*"west" + 0.013*"price" + 0.013*"gold"

Score: 0.0

In [46]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.6012848615646362	 
Topic: 0.012*"news" + 0.006*"farm" + 0.006*"coal" + 0.006*"monday" + 0.006*"farmer" + 0.006*"energi" + 0.006*"queensland" + 0.005*"dairi" + 0.005*"rain" + 0.005*"busi"

Score: 0.23870429396629333	 
Topic: 0.016*"crash" + 0.013*"podcast" + 0.010*"turnbul" + 0.009*"die" + 0.009*"stab" + 0.008*"michael" + 0.007*"fatal" + 0.006*"truck" + 0.006*"dead" + 0.006*"kill"

Score: 0.020003337413072586	 
Topic: 0.016*"rural" + 0.016*"countri" + 0.014*"hour" + 0.008*"govern" + 0.008*"health" + 0.007*"fund" + 0.006*"grandstand" + 0.005*"sport" + 0.005*"nation" + 0.005*"servic"

Score: 0.020001590251922607	 
Topic: 0.017*"charg" + 0.017*"polic" + 0.015*"murder" + 0.011*"court" + 0.010*"death" + 0.010*"woman" + 0.009*"jail" + 0.008*"child" + 0.008*"assault" + 0.008*"accus"

Score: 0.020001519471406937	 
Topic: 0.018*"trump" + 0.011*"market" + 0.009*"donald" + 0.008*"weather" + 0.008*"share" + 0.006*"australian" + 0.006*"street" + 0.006*"tuesday" + 0.006*"live" + 0.006*"wall

In [47]:
###Unseen document
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3499999940395355	 Topic: 0.031*"charg" + 0.030*"court" + 0.024*"polic" + 0.023*"murder" + 0.019*"face"
Score: 0.349981427192688	 Topic: 0.027*"elect" + 0.023*"south" + 0.022*"nation" + 0.016*"open" + 0.014*"australia"
Score: 0.18333333730697632	 Topic: 0.017*"brisban" + 0.016*"famili" + 0.014*"child" + 0.013*"report" + 0.012*"jail"
Score: 0.016680026426911354	 Topic: 0.035*"trump" + 0.022*"kill" + 0.021*"north" + 0.020*"coast" + 0.016*"china"
Score: 0.016671927645802498	 Topic: 0.027*"govern" + 0.017*"plan" + 0.014*"rural" + 0.013*"say" + 0.013*"council"
Score: 0.01666666753590107	 Topic: 0.032*"queensland" + 0.020*"countri" + 0.018*"hospit" + 0.017*"tasmanian" + 0.017*"hour"
Score: 0.01666666753590107	 Topic: 0.030*"australia" + 0.023*"world" + 0.017*"market" + 0.014*"sydney" + 0.012*"record"
Score: 0.01666666753590107	 Topic: 0.017*"tasmania" + 0.015*"time" + 0.015*"fight" + 0.013*"lose" + 0.012*"leagu"
Score: 0.01666666753590107	 Topic: 0.023*"hous" + 0.019*"die" + 0.016*"h